In [1]:
# Collected data from Lisa/SLURM using the following command:
# sacct -a --starttime 2022-10-01 --endtime 2022-10-31 --format=jobid,gid,uid,partition,submit,start,end,elapsedraw,cputimeraw,ncpus,nnodes,nodelist,exitcode,state,AllocCPUS,AllocNodes,AveCPUFreq,AveDiskRead,AveDiskWrite,MaxDiskRead,MaxDiskWrite,MaxVMSize,NTasks,ReqCPUS,ReqMem > $HOME/slurm_data.csv
# sacct -a --starttime 2022-01-01 --endtime 2022-12-31 --format=jobid,gid,uid,partition,submit,start,end,elapsedraw,cputimeraw,ncpus,nnodes,nodelist,exitcode,state,AllocCPUS,AllocNodes,AveCPUFreq,AveDiskRead,AveDiskWrite,MaxDiskRead,MaxDiskWrite,MaxVMSize,NTasks,ReqCPUS,ReqMem > $HOME/slurm_data_2022.csv

# Required preprocessing/parsing of the job data

import os
import numpy as np
import pandas as pd
from multiprocessing import cpu_count
from pandarallel import pandarallel
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import time, datetime, pytz
from matplotlib.ticker import MultipleLocator, FixedLocator, LogLocator, NullFormatter, ScalarFormatter
from datetime import date, datetime, time, timedelta
import re

In [2]:
# Load and preprocess raw slurmdata collected from SLURM:
#   convert data tpye -> int 
#   Unknown -> NaN values

def preprocess_df(file_name):
    with open(os.path.join(file_path, file_name),'r') as file:
        filedata = file.read()
        filedata = filedata.replace('None assigned','NoneAssigned')
    # with open(os.path.join(location_job_data_csv, str('processed_'+name)),'w') as file:
    #     file.write(filedata)
    #data = pd.read_fwf(os.path.join(file_path, file_name), delimiter=r"\s+", header=None)#, low_memory=False)
    data = pd.read_fwf(os.path.join(file_path, file_name), delimiter=r"\s", header=None)
    data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    data = data.rename(columns=data.iloc[0]).drop(data.index[0])
    data = data.iloc[1:]
    data = data.astype({"ElapsedRaw": int, "CPUTimeRAW": int, "NCPUS": int, "NNode":int, "AllocCPUS":int, "AllocNode":int, "ReqCPUS":int})
    data.replace(to_replace=r'^\s*$', value=np.nan, regex=True, inplace=True)  # Blank value->NaN value
    data.replace(to_replace=r'Unknown', value=np.nan, regex=True, inplace=True)  # Unknown value->NaN value
    return(data)

file_path = "/Users/chuxiaoyu/VScodeProjects/SURFJob/"
data = preprocess_df("slurm_data_2022.csv")

data

,JobID,GID,UID,Partition,Submit,Start,End,ElapsedRaw,CPUTimeRAW,NCPUS,...,AllocNode,AveCPUFreq,AveDiskRead,AveDiskWrite,MaxDiskRead,MaxDiskWrite,MaxVMSize,NTask,ReqCPUS,ReqMem
2,8619864,54655,55101,gpu_titan+,2022-01-01T01:55:55,2022-01-01T01:55:55,2022-01-01T02:02:51,416,9984,24,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,186G
3,8619865,54655,55101,gpu_titan+,2022-01-01T03:49:53,2022-01-01T03:50:05,2022-01-01T03:56:51,406,9744,24,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,186G
4,8619866,54655,55101,gpu_titan+,2022-01-01T04:19:20,2022-01-01T04:19:20,2022-01-01T04:29:50,630,15120,24,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,186G
5,8619867,54655,55101,gpu_titan+,2022-01-01T04:32:24,2022-01-01T04:32:24,2022-01-01T04:42:22,598,14352,24,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,186G
6,8619868,54655,55101,gpu_titan+,2022-01-01T04:47:23,2022-01-01T04:47:23,2022-01-01T04:50:11,168,4032,24,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,186G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301705,10587253,47200,47539,normal,2022-12-31T22:44:37,2023-01-01T04:16:28,2023-01-01T04:19:54,206,3296,16,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5760M
2301706,10587254,47200,47539,normal,2022-12-31T22:44:37,2023-01-01T04:17:59,2023-01-01T04:22:00,241,3856,16,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5760M
2301707,10587255,59552,60028,shared,2022-12-31T23:05:42,2022-12-31T23:05:42,2022-12-31T23:12:55,433,866,2,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,8610M
2301708,10587255.ba+,NaN,NaN,NaN,2022-12-31T23:05:42,2022-12-31T23:05:42,2022-12-31T23:12:55,433,866,2,...,1,1M,96.08M,0.86M,96.08M,0.86M,790132K,1,2,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2301708 entries, 2 to 2301709
Data columns (total 25 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   JobID         object
 1   GID           object
 2   UID           object
 3   Partition     object
 4   Submit        object
 5   Start         object
 6   End           object
 7   ElapsedRaw    int64 
 8   CPUTimeRAW    int64 
 9   NCPUS         int64 
 10  NNode         int64 
 11  NodeList      object
 12  ExitCode      object
 13  State         object
 14  AllocCPUS     int64 
 15  AllocNode     int64 
 16  AveCPUFreq    object
 17  AveDiskRead   object
 18  AveDiskWrite  object
 19  MaxDiskRead   object
 20  MaxDiskWrite  object
 21  MaxVMSize     object
 22  NTask         object
 23  ReqCPUS       int64 
 24  ReqMem        object
dtypes: int64(7), object(18)
memory usage: 456.6+ MB


In [4]:
# Process NaN values

data = data.dropna(subset=["Start", "End"])  # drop NaN values in Start, End

In [5]:
# Unify data: 
# ->AveCPUFreq M/K-> M, 
#   AveDiskRead -> M, 
#   AveDiskWrite -> M, 
#   MaxDiskRead -> M, 
#   MaxDiskWrite -> M, 
#   MaxVMSize -> M, 
# ->ReqMem M/G-> M,
# ToDo\\->NodeList -> r1n[1-3], r2n[4-5] sepereate to a list: [r1n1, r1n2, r1n3, r2n4, r2n5]

def unify_ave_cpu_freq(df):
    ave_cpu_freq_l = []
    for i in df["AveCPUFreq"]:
        if pd.isnull(i):
            ave_cpu_freq_num = i
        else:
            ave_cpu_freq = re.findall(r"\d+\.?\d*", i)
            ave_cpu_freq_num = ave_cpu_freq[0]
            if 'K' in i:
                ave_cpu_freq_num = format(float(ave_cpu_freq_num)/1024.00, '.2f')
            else:
                ave_cpu_freq_num = format(float(ave_cpu_freq_num)/1.00, '.2f')
        ave_cpu_freq_l.append(ave_cpu_freq_num)
    df["ave_cpu_freq"] = ave_cpu_freq_l


def unify_req_mem(df):
    req_mem_l = []
    for i in df["ReqMem"]:
        if pd.isnull(i):
            req_mem_num = i
        else:
            req_mem = re.findall(r"\d+\.?\d*", i)
            req_mem_num = req_mem[0]
            if 'G' in i:
                req_mem_num = format(float(req_mem_num)*1024.00, '.2f')
            else:
                req_mem_num = format(float(req_mem_num)*1.00, '.2f')
        req_mem_l.append(req_mem_num)
    df["req_mem"] = req_mem_l

    
# parses node strings like r12n[1-30,32] to r12n1, r12n2 ... r12n30, r12n32    
def split_nodes(s):
    if s is None or len(s) == 0:
        return set()
    
    s = s.replace("\r\n", "").replace("\n", "").replace("\t", "")

    start = 0
    index = 0
    rack_chunks = []
    in_bracket = False
    while index < len(s):  # Separate them in parts like r12n[1-30,32] or r13n1
        if s[index] == "[":
            in_bracket = True
        elif s[index] == "]":
            in_bracket = False
        elif s[index] == "," and not in_bracket:
            rack_chunks.append(s[start: index])
            start = index + 1
        index += 1
    rack_chunks.append(s[start: index])  # Add the last line
    
    node_names = set()

    for rack_chunk in rack_chunks:
        if "[" in rack_chunk:
            prefix, postfix = rack_chunk.split("[")
            postfix = postfix[:-1]  # Remove the last bracket
            nodes = postfix.split(",")
            for node in nodes:
                if "-" in node:
                    start, end = node.split("-")
                    if not start.isnumeric() or not end.isnumeric():
                        continue
                    for i in range(int(start), int(end) + 1):
                        node_names.add("{}{}".format(prefix, i))
                else:
                    node_names.add("{}{}".format(prefix, node))
        else:
            node_names.add(rack_chunk)

    return node_names


# Creates a column that indicates if a job was executed on ML nodes
gpu_nodes = {
    "r28n1", "r28n2", "r28n3", "r28n4", "r28n5",
    "r29n1", "r29n2", "r29n3", "r29n4", "r29n5",
    "r30n1", "r30n2", "r30n3", "r30n4", "r30n5", "r30n6", "r30n7",
    "r31n1", "r31n2", "r31n3", "r31n4", "r31n5", "r31n6"
    "r32n1", "r32n2", "r32n3", "r32n4", "r32n5", "r32n6", "r32n7",
    "r33n2", "r33n3", "r33n5", "r33n6",
    "r34n1", "r34n2", "r34n3", "r34n4", "r34n5", "r34n6", "r34n7",
    "r35n1", "r35n2", "r35n3", "r35n4", "r35n5",
    "r36n1", "r36n2", "r36n3", "r36n4", "r36n5",
    "r38n1", "r38n2", "r38n3", "r38n4", "r38n5",
}

def calculate_perjob(row):
    nodes = row["NodeList"]
    splitnodes = split_nodes(nodes)
    return any([n in gpu_nodes for n in splitnodes])

pandarallel.initialize(nb_workers=min(cpu_count(), 8), progress_bar=True)
data["is_gpu"] =  data.parallel_apply(calculate_perjob, axis=1)


def unify_state(df):
    state_l = []
    for i in df['State']:
        if i == "CANCELLED+":
            i = "CANCELLED"
        state_l.append(i)
    df['state'] = state_l
    
def unify_data(df):
    unify_ave_cpu_freq(df)
    unify_req_mem(df)
    unify_state(df)
    l = []
    for s in df["NodeList"]:
        l.append(split_nodes(s))
    df["node_list"] = l
    
unify_data(data)
data.head(5)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


<ipython-input-5-32bcd6ff7f79>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["is_gpu"] =  data.parallel_apply(calculate_perjob, axis=1)
<ipython-input-5-32bcd6ff7f79>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ave_cpu_freq"] = ave_cpu_freq_l
<ipython-input-5-32bcd6ff7f79>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

,JobID,GID,UID,Partition,Submit,Start,End,ElapsedRaw,CPUTimeRAW,NCPUS,...,MaxDiskWrite,MaxVMSize,NTask,ReqCPUS,ReqMem,is_gpu,ave_cpu_freq,req_mem,state,node_list
2,8619864,54655,55101,gpu_titan+,2022-01-01T01:55:55,2022-01-01T01:55:55,2022-01-01T02:02:51,416,9984,24,...,NaN,NaN,NaN,20,186G,True,NaN,190464.00,COMPLETED,{r28n1}
3,8619865,54655,55101,gpu_titan+,2022-01-01T03:49:53,2022-01-01T03:50:05,2022-01-01T03:56:51,406,9744,24,...,NaN,NaN,NaN,20,186G,True,NaN,190464.00,COMPLETED,{r28n1}
4,8619866,54655,55101,gpu_titan+,2022-01-01T04:19:20,2022-01-01T04:19:20,2022-01-01T04:29:50,630,15120,24,...,NaN,NaN,NaN,20,186G,True,NaN,190464.00,TIMEOUT,{r28n1}
5,8619867,54655,55101,gpu_titan+,2022-01-01T04:32:24,2022-01-01T04:32:24,2022-01-01T04:42:22,598,14352,24,...,NaN,NaN,NaN,20,186G,True,NaN,190464.00,CANCELLED,{r28n1}
6,8619868,54655,55101,gpu_titan+,2022-01-01T04:47:23,2022-01-01T04:47:23,2022-01-01T04:50:11,168,4032,24,...,NaN,NaN,NaN,20,186G,True,NaN,190464.00,COMPLETED,{r28n1}


In [6]:
# Construct new fields: 
#   Submit, Start, End
#   submit_hour_of_day, submit_day_of_week, submit_date
#   waiting_time=Start-Submit, running_time=End-Start

data['Submit'] = pd.to_datetime(data['Submit'], utc=False, format="%Y-%m-%dT%H:%M:%S")
data['Start'] = pd.to_datetime(data['Start'], utc=False, format="%Y-%m-%dT%H:%M:%S")
data['End'] = pd.to_datetime(data['End'], utc=False, format="%Y-%m-%dT%H:%M:%S")

data["submit_hour_of_day"] = data["Submit"].dt.hour
data["submit_day_of_week"] = data["Submit"].apply(lambda x : x.weekday())
data["submit_date"] = data["Submit"].dt.date

data["waiting_time"] = data["Start"] - data["Submit"]
data["waiting_time"] = data["waiting_time"].apply(lambda x:timedelta.total_seconds(x))

# running_time = ElapsedRaw
# data["running_time"] = data["End"] - data["Start"]
# data["running_time"] = data["running_time"].apply(lambda x:timedelta.total_seconds(x))

data.head()

<ipython-input-6-be9acc1ab964>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Submit'] = pd.to_datetime(data['Submit'], utc=False, format="%Y-%m-%dT%H:%M:%S")
<ipython-input-6-be9acc1ab964>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Start'] = pd.to_datetime(data['Start'], utc=False, format="%Y-%m-%dT%H:%M:%S")
<ipython-input-6-be9acc1ab964>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,JobID,GID,UID,Partition,Submit,Start,End,ElapsedRaw,CPUTimeRAW,NCPUS,...,ReqMem,is_gpu,ave_cpu_freq,req_mem,state,node_list,submit_hour_of_day,submit_day_of_week,submit_date,waiting_time
2,8619864,54655,55101,gpu_titan+,2022-01-01 01:55:55,2022-01-01 01:55:55,2022-01-01 02:02:51,416,9984,24,...,186G,True,NaN,190464.00,COMPLETED,{r28n1},1,5,2022-01-01,0.0
3,8619865,54655,55101,gpu_titan+,2022-01-01 03:49:53,2022-01-01 03:50:05,2022-01-01 03:56:51,406,9744,24,...,186G,True,NaN,190464.00,COMPLETED,{r28n1},3,5,2022-01-01,12.0
4,8619866,54655,55101,gpu_titan+,2022-01-01 04:19:20,2022-01-01 04:19:20,2022-01-01 04:29:50,630,15120,24,...,186G,True,NaN,190464.00,TIMEOUT,{r28n1},4,5,2022-01-01,0.0
5,8619867,54655,55101,gpu_titan+,2022-01-01 04:32:24,2022-01-01 04:32:24,2022-01-01 04:42:22,598,14352,24,...,186G,True,NaN,190464.00,CANCELLED,{r28n1},4,5,2022-01-01,0.0
6,8619868,54655,55101,gpu_titan+,2022-01-01 04:47:23,2022-01-01 04:47:23,2022-01-01 04:50:11,168,4032,24,...,186G,True,NaN,190464.00,COMPLETED,{r28n1},4,5,2022-01-01,0.0


In [61]:
# Split datasets into job_data and task_data

def transfer_job_id(df):
    job_ids = []
    for JobID in df:
        JobID_l = re.findall(r"\d+", JobID)
        job_id = ''
        for i in JobID_l:
            job_id = job_id + i
        if '.' in JobID or '+' in JobID:
            job_id = job_ids[-1]
        job_ids.append(job_id)
    return job_ids

def create_task_id(df):
    task_ids = [0]

    task_count = 0
    
    for i in range(len(df)-1):
        last = df.iloc[i]
        curr = df.iloc[i+1]
        if int(curr) == int(last):
            task_count += 1
            task_ids.append(task_count)
        else:
            task_count = 0
            task_ids.append(task_count)
    return task_ids


def split_job_and_task(df):
    job_data = data.loc[data["task_id"]==0]
    task_data = data.loc[data["task_id"]!=0]
    return job_data, task_data

data["job_id"] = transfer_job_id(data["JobID"])
data["task_id"] = create_task_id(data["job_id"])
job_data, task_data = split_job_and_task(data)


<ipython-input-61-7795a041d999>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["job_id"] = transfer_job_id(data["JobID"])
<ipython-input-61-7795a041d999>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["task_id"] = create_task_id(data["job_id"])


In [62]:
# data = data.loc[:, ]
job_data = job_data.loc[:, ["job_id","JobID","GID","UID","Partition","Submit","Start","End","ElapsedRaw","CPUTimeRAW","NCPUS","NNode","state","AllocCPUS","AllocNode","ReqCPUS","req_mem","node_list","submit_hour_of_day","submit_day_of_week","submit_date","waiting_time","is_gpu"]]

In [7]:
# Save slurm_data, job_data, and task_data as .csv files

data.to_csv(file_path + "slurm_data_2022_cleaned.csv")
# job_data.to_csv(file_path + "job_data_cleaned.csv")
# task_data.to_csv(file_path + "task_data_cleaned.csv")

In [55]:
job_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158954 entries, 2 to 471197
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   job_id              158954 non-null  object        
 1   JobID               158954 non-null  object        
 2   GID                 158954 non-null  object        
 3   UID                 158954 non-null  object        
 4   Partition           158954 non-null  object        
 5   Submit              158954 non-null  datetime64[ns]
 6   Start               158954 non-null  datetime64[ns]
 7   End                 158954 non-null  datetime64[ns]
 8   ElapsedRaw          158954 non-null  int64         
 9   CPUTimeRAW          158954 non-null  int64         
 10  NCPUS               158954 non-null  int64         
 11  NNode               158954 non-null  int64         
 12  State               158954 non-null  object        
 13  AllocCPUS           158954 no

In [56]:
task_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312239 entries, 30 to 471195
Data columns (total 35 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   JobID               312239 non-null  object        
 1   GID                 478 non-null     object        
 2   UID                 478 non-null     object        
 3   Partition           478 non-null     object        
 4   Submit              312239 non-null  datetime64[ns]
 5   Start               312239 non-null  datetime64[ns]
 6   End                 312239 non-null  datetime64[ns]
 7   ElapsedRaw          312239 non-null  int64         
 8   CPUTimeRAW          312239 non-null  int64         
 9   NCPUS               312239 non-null  int64         
 10  NNode               312239 non-null  int64         
 11  NodeList            312239 non-null  object        
 12  ExitCode            312113 non-null  object        
 13  State               312239 n